In [1]:
import pandas as pd

path = input("Type in path to csv: ")
df = pd.read_csv (path)
FAdf = pd.read_csv (r'/content/free_agents.csv')

Type in path to csv: /content/981c6fa8-615e-44fb-9eb6-58c4b220d12e_2022-02-19.csv


,Picked At,Pick Number,Appearance,First Name,Last Name,Team,Position,Draft,Draft Entry,Draft Entry Fee,Draft Size,Draft Total Prizes,Tournament Title,Tournament,Tournament Entry Fee,Tournament Total Prizes,Tournament Size,Draft Pool Title,Draft Pool,Draft Pool Entry Fee,Draft Pool Total Prizes,Draft Pool Size
0,2022-02-15 23:52:57 UTC,173,229a7325-5a1c-4eca-8cb1-2ef53683316e,Malik,Willis,NaN,QB,5cf99568-7439-40da-ade9-7cdcf175e931,f3acd242-d21e-4aa7-8cc5-c8ee0e36efc5,0.0,12,0.0,The Big Board,f9aed24a-d17e-451f-a280-026b656c2190,10.0,250000.0,28224,NaN,NaN,NaN,NaN,NaN
1,2022-02-15 23:45:56 UTC,140,82b31123-b97b-4839-91c2-d0329c7f2528,Tua,Tagovailoa,MIA,QB,5cf99568-7439-40da-ade9-7cdcf175e931,f3acd242-d21e-4aa7-8cc5-c8ee0e36efc5,0.0,12,0.0,The Big Board,f9aed24a-d17e-451f-a280-026b656c2190,10.0,250000.0,28224,NaN,NaN,NaN,NaN,NaN
2,2022-02-15 23:37:15 UTC,92,b0f885dd-a5ef-49ea-8ab1-434fe3bf9af2,Jalen,Hurts,PHI,QB,5cf99568-7439-40da-ade9-7cdcf175e931,f3acd242-d21e-4aa7-8cc5-c8ee0e36efc5,0.0,12,0.0,The Big Board,f9aed24a-d17e-451f-a280-026b656c2190,10.0,250000.0,28224,NaN,NaN,NaN,NaN,NaN
3,2022-02-16 00:47:11 UTC,197,eb8b21a7-cb48-4d57-96b4-35e33dd28b4a,Jared,Goff,DET,QB,0ba88331-46ae-49ef-9c4a-b6c32c6bc148,c9e8aae4-b43b-4657-b3de-4658772a3379,0.0,12,0.0,The Big Board,f9aed24a-d17e-451f-a280-026b656c2190,10.0,250000.0,28224,NaN,NaN,NaN,NaN,NaN
4,2022-02-16 00:11:55 UTC,29,5f756a9f-c7d1-42f2-94f1-66e3502c03da,Josh,Allen,BUF,QB,0ba88331-46ae-49ef-9c4a-b6c32c6bc148,c9e8aae4-b43b-4657-b3de-4658772a3379,0.0,12,0.0,The Big Board,f9aed24a-d17e-451f-a280-026b656c2190,10.0,250000.0,28224,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
415,2022-02-18 17:50:04 UTC,32,5cc83aaa-973b-4988-b1ee-eb920d32bb4b,Kyle,Pitts,ATL,TE,2e5a73de-1c83-4a00-8ac0-0cf33c200cda,2d64b205-c6b5-4cec-bfb9-1a750a329e49,0.0,12,0.0,The Big Board,f9aed24a-d17e-451f-a280-026b656c2190,10.0,250000.0,28224,NaN,NaN,NaN,NaN,NaN
416,2022-02-18 20:18:02 UTC,198,24455553-bbb2-4050-8e58-996e844b46af,Donald,Parham,LAC,TE,e63ee71f-7516-4a4b-ae9f-05ef6566b72f,5c07c090-b7c9-4715-b257-8628c98fbe4a,0.0,12,0.0,The Big Board,f9aed24a-d17e-451f-a280-026b656c2190,10.0,250000.0,28224,NaN,NaN,NaN,NaN,NaN
417,2022-02-18 19:37:10 UTC,30,5cc83aaa-973b-4988-b1ee-eb920d32bb4b,Kyle,Pitts,ATL,TE,e63ee71f-7516-4a4b-ae9f-05ef6566b72f,5c07c090-b7c9-4715-b257-8628c98fbe4a,0.0,12,0.0,The Big Board,f9aed24a-d17e-451f-a280-026b656c2190,10.0,250000.0,28224,NaN,NaN,NaN,NaN,NaN
418,2022-02-18 21:03:56 UTC,135,1c48d136-49bc-4ba8-8e77-496246658345,Cole,Kmet,CHI,TE,be2b2c34-bce5-4c76-8014-26ac89487917,2e5be26f-f55a-4eb4-b55d-e78c662b90d5,0.0,12,0.0,The Big Board,f9aed24a-d17e-451f-a280-026b656c2190,10.0,250000.0,28224,NaN,NaN,NaN,NaN,NaN


In [2]:
def GetFreeAgents(df):
  freeAgentList = []
  for i in range(len(df)):
    player = Player(df.at[i, 'First Name'], df.at[i, 'Last Name'], df.at[i, 'Position'], df.at[i, 'Team'], -1)
    freeAgentList.append(player.playerkey)
  return freeAgentList

In [34]:
from operator import itemgetter
class Player:
  def __init__(self, firstname, lastname, pos, team, picknum):
    self.firstname = firstname
    self.lastname = lastname
    self.pos = pos
    self.team = team
    self.playerkey = '{}_{}_{}_{}'.format(self.firstname, self.lastname, self.pos, self.team)
    self.drafts = []
    self.common_expo = []
    self.picknums = [picknum]
  def __repr__(self):
    return '{} {}'.format(self.firstname, self.lastname)
  def AddPick(self, pick):
    self.picknums.append(pick)
  def getPicks(self):
    return self.picknums
  def AddExpo(self, df, num, freeAgents):
    self.drafts.append(num)
    for i in range(len(df)):
      player = Player(df.at[i, 'First Name'], df.at[i, 'Last Name'], df.at[i, 'Position'], df.at[i, 'Team'], df.at[i, 'Pick Number'])
      if self.playerkey != player.playerkey:
        newExpo = True
        for j in self.common_expo:
          if j[0].playerkey == player.playerkey:
            j[3] += 1
            j[4].append(num)
            newExpo = False
            break
        if newExpo == True:
          if player.playerkey in freeAgents:
              player.team = '{}*'.format(player.team)
          self.common_expo.append([player,player.pos,player.team, 1, [num]])
  def GetCommonExpo(self):
    return pd.DataFrame(sorted(self.common_expo, key=itemgetter(3), reverse=True), columns = ['Player','Pos','Team','Times Drafted Together','Common Drafts'])
  def GetPlayerInfo(self):
    print('{} {} {} {} {}\n'.format(self.firstname, self.lastname, self.pos, self.team, self.picknums))
    return pd.DataFrame(sorted(self.common_expo, key=itemgetter(3), reverse=True), columns = ['Player','Pos','Team','Times Drafted Together','Common Drafts'])


In [36]:
def DraftGen(df):
  df2 = df.sort_values(by=['Draft']).iloc[:, 0:8]
  drafts = []
  for i in range(len(df)):
     if df2.at[i, 'Draft'] not in drafts:
       drafts.append(df2.at[i, 'Draft'])
  for i in range(len(df)):
    for k in range(len(drafts)):
      if df2.at[i, 'Draft'] == drafts[k]:
        df2.at[i, 'Draft'] = k + 1
  drafts = dict(iter(df2.groupby('Draft')))
  return drafts

In [42]:
def AddDraft(df, playerlist, num, freeAgents):
  for i in range(len(df)):
    player = Player(df.at[i, 'First Name'], df.at[i, 'Last Name'], df.at[i, 'Position'], df.at[i, 'Team'], df.at[i, 'Pick Number'])
    newPlayer = True
    for j in playerlist:
      if j.playerkey == player.playerkey:
        j.AddExpo(df,num, freeAgents)
        j.AddPick(df.at[i, 'Pick Number'])
        newPlayer = False
        break
    if newPlayer == True:
      if player.playerkey in freeAgents:
        player.team = '{}*'.format(player.team)
      playerlist.append(player)
      player.AddExpo(df,num, freeAgents)
  return playerlist

In [38]:
def PlayerSearch(playerList, search):
  #TODO If players have same name
  searchedPlayer = search.split()
  retList = [player for player in playerList if player.firstname == searchedPlayer[0] and player.lastname == searchedPlayer[1]]
  return retList

In [39]:
def Search():
  search = input("Search: ")
  player = PlayerSearch(playerList, search)
  return player[0].GetPlayerInfo()

In [35]:
freeAgents = GetFreeAgents(FAdf)

In [40]:
drafts = DraftGen(df)

In [55]:
playerList = []
for i in drafts:
  drafts[i] = drafts.get(i).sort_values(by=['Picked At'])
  playerList = AddDraft(drafts.get(i).reset_index(), playerList, i, freeAgents)

In [56]:
Search()

Search: Josh Allen
Josh Allen QB BUF [29, 31]



,Player,Pos,Team,Times Drafted Together,Common Drafts
0,Austin Ekeler,RB,LAC,2,"[2, 7]"
1,Tyler Boyd,WR,CIN,2,"[2, 7]"
2,Isaiah McKenzie,WR,BUF*,2,"[2, 7]"
3,Mark Andrews,TE,BAL,1,[2]
4,Elijah Moore,WR,NYJ,1,[2]
5,Amon-Ra St. Brown,WR,DET,1,[2]
6,Gabriel Davis,WR,BUF,1,[2]
7,AJ Dillon,RB,GB,1,[2]
8,James Conner,RB,ARI*,1,[2]
9,Corey Davis,WR,NYJ,1,[2]


In [57]:
drafts[6]

,Picked At,Pick Number,Appearance,First Name,Last Name,Team,Position,Draft
86,2022-02-16 14:55:24 UTC,12,06f02542-09e0-485f-8638-d2baf314087e,Dalvin,Cook,MIN,RB,6
85,2022-02-16 14:55:27 UTC,13,e1ee2f83-6097-4b00-98d7-f860bbf1b0aa,Joe,Mixon,CIN,RB,6
227,2022-02-16 14:59:35 UTC,36,76b19c78-3e05-4892-8d9d-d14e05a6eb00,Mike,Evans,TB,WR,6
226,2022-02-16 14:59:58 UTC,37,7866fc72-c7c8-4936-8644-279df3a4ffb0,Calvin,Ridley,ATL,WR,6
225,2022-02-16 15:04:41 UTC,60,9d418b14-6bcb-4dd1-ad5b-720d2c047f33,Hunter,Renfrow,LV,WR,6
14,2022-02-16 15:04:51 UTC,61,e084c2e7-cfd2-4d29-a018-dc6a61bd9eec,Kyler,Murray,ARI,QB,6
224,2022-02-16 15:10:53 UTC,84,79bb2693-7255-410a-84de-f6f57aa94a4a,Adam,Thielen,MIN,WR,6
84,2022-02-16 15:11:18 UTC,85,43538f36-29e7-47b0-9a20-70a2b1bd8a06,James,Conner,ARI,RB,6
223,2022-02-16 15:15:14 UTC,108,6eefcd38-448d-42be-a38b-5d7aeedee8da,Christian,Kirk,ARI,WR,6
383,2022-02-16 15:15:35 UTC,109,8f2f9248-0acb-43a7-a694-39cc03f8246f,Zach,Ertz,ARI,TE,6
